In [1]:
import json 
# data\DCM\DCM_json\03\03_regulation_main.json
with open("C:/Users/Shic/Downloads/legal_graph/data/DCM/DCM_json/04/04_regulation_main.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

len(data)

6173

In [7]:
t = "회사 및 임직원은 선량한 관리자의 주의로서 투자자의 매매주문, 배당 및 투자자예탁금 이용료의 입금ㆍ고, 투자자의 각종 권리행사 등에 따른 업무처리가 공정하고 정확하게 이루어질 수 있도록 맡은 업무를 충실하게 수행하여야 한다."

len(t)

125

In [3]:
yakgwan = ["매매거래계좌설정약관",
"파생상품계좌설정약관",
"외화증권매매거래계좌설정약관",
"신용거래약관",
"증권 대차거래 약관",
"연금저축계좌설정약관",
"대고객환매조건부매매약관",
"기관간환매조건부매매약관",
"기업어음증권거래약관",
"전자금융거래 이용에 관한 기본약관",
"외국집합투자증권 매매거래에 관한 표준약관",
"수익증권저축약관",
"해외파생상품시장거래중개계좌설정약관",
"해외파생상품시장거래총괄계좌설정약관",
"유사해외통화선물(FX Margin Trading) 거래 약관"]

In [4]:
idx_list = []

In [5]:
for idx, text in enumerate(data):
    if text['metadata']['document_title'] in yakgwan:
        idx_list.append(idx)

len(idx_list)

913

In [ ]:
import json 
# data\DCM\DCM_json\03\03_regulation_main.json
with open("C:/Users/Shic/Downloads/legal_graph/data/DCM/DCM_json/04/04_regulation_main.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

len(data)

In [2]:
# 한자 preprocess test
import codes.GraphDB.process as preprocess

text = ["法律을 준수해야 한다.", "최근 金錢적으로 어려움을 겪는 사람이 늘고 있다",
        "이 문장에는 한자(漢字)가 포함되어 있습니다.",
        "한자(漢字)를 제거하고 싶습니다.",
        "괄호 없이 漢字만 있는 경우도 처리합니다.",
        "한자와 (漢字) 괄호를 모두 제거합니다."]
print(preprocess.replace_hanja(text))

['을 준수해야 한다.', '최근 적으로 어려움을 겪는 사람이 늘고 있다', '이 문장에는 한자가 포함되어 있습니다.', '한자를 제거하고 싶습니다.', '괄호 없이 만 있는 경우도 처리합니다.', '한자와  괄호를 모두 제거합니다.']


In [ ]:
from neo4j import GraphDatabase
import json

# Neo4j 데이터베이스에 연결합니다.
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'password'))

def flatten_properties(node_dict):
    props = {}
    for key, value in node_dict.items():
        if isinstance(value, (dict, list)):
            # dict나 list를 JSON 문자열로 변환합니다.
            props[key] = json.dumps(value, ensure_ascii=False)
        else:
            props[key] = value
    return props

def create_triplet(tx, node1_dict, rel_type, node2_dict):
    node1_props = flatten_properties(node1_dict)
    node2_props = flatten_properties(node2_dict)

    # 노드 1을 생성하거나 병합합니다.
    tx.run(
        '''
        MERGE (a:Clause {index: $index1})
        SET a += $props1
        ''',
        index1=node1_dict['index'][0],
        props1=node1_props
    )

    # 노드 2를 생성하거나 병합합니다.
    tx.run(
        '''
        MERGE (b:Clause {index: $index2})
        SET b += $props2
        ''',
        index2=node2_dict['index'][0],
        props2=node2_props
    )

    # 관계 유형이 유효한지 확인합니다.
    if not rel_type.isidentifier():
        raise ValueError("Invalid relationship type")

    # 노드 사이에 관계를 생성합니다.
    tx.run(
        '''
        MATCH (a:Clause {index: $index1}), (b:Clause {index: $index2})
        MERGE (a)-[:%s]->(b)
        ''' % rel_type,
        index1=node1_dict['index'][0],
        index2=node2_dict['index'][0]
    )

# 제공된 트리플릿 데이터
triplet = [
    {
        'index': ['제125조제1항', '제119조제3항'],
        'subtitle': ['거짓의 기재 등으로 인한 배상책임', '모집 또는 매출의 신고'],
        'content': '증권신고서(정정신고서 및 첨부서류를 포함한다...)',
        'metadata': {
            'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )/n',
            'date': '시행 2024. 8. 14.',
            'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정/n',
            'source': '국가법령정보센터',
            'title': {
                'doc': ['제3편 증권의 발행 및 유통', '제3편 증권의 발행 및 유통'],
                'chapter': ['제1장 증권신고서', '제1장 증권신고서'],
                'section': ['제2절 금융투자업자별 영업행위 규칙', '제2절 금융투자업자별 영업행위 규칙'],
                'subsection': ['제4관 신탁업자의 영업행위 규칙', '제4관 신탁업자의 영업행위 규칙'],
                'supplementary': None
            }
        },
        'keyword_name': '증권신고서'
    },
    'INCLUDE',
    {
        'index': ['제125조제1항'],
        'subtitle': ['거짓의 기재 등으로 인한 배상책임'],
        'content': '증권신고서(정정신고서 및 첨부서류를 포함한다...)',
        'metadata': {
            'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )/n',
            'date': '시행 2024. 8. 14.',
            'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정/n',
            'source': '국가법령정보센터',
            'title': {
                'doc': ['제3편 증권의 발행 및 유통'],
                'chapter': ['제1장 증권신고서'],
                'section': ['제2절 금융투자업자별 영업행위 규칙'],
                'subsection': ['제4관 신탁업자의 영업행위 규칙'],
                'supplementary': None
            }
        },
        'keyword_name': '정정신고서'
    }
]

triplets = [triplet]  # 트리플릿 리스트

# 세션을 열고 트리플릿을 처리합니다.
with driver.session() as session:
    for triplet in triplets:
        node1_dict, rel_type, node2_dict = triplet
        session.write_transaction(create_triplet, node1_dict, rel_type, node2_dict)


# 2개의 키워드 같이 찾기

In [ ]:
import json

def print_clause(clauses: list):
    for clause in clauses:
        print(f'{clause["index"]} : {clause["content"]}')

def search_json(json_path, word1, word2):
    clauses_and = []
    clauses_1 = []
    clauses_2 = []

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for clause in data:
        if word1 in clause["content"] and word2 in clause["content"]:
            clauses_and.append(clause)
        if word1 in clause["content"]:
            clauses_1.append(clause)
        if word2 in clause["content"]:
            clauses_2.append(clause)
    
    print(f'## {word1} & {word2} ##')
    if len(clauses_and) > 0:
        print_clause(clauses_and)
    else:
        print("공통조항 없음")
    
    print(f'/n/n## {word1} 언급 조항 ##')
    if len(clauses_1) > 0:
        print_clause(clauses_1)
    else:
        print(f"{word1} 언급 조항 없음")
    
    print(f'/n/n## {word2} 언급 조항 ##')
    if len(clauses_2) > 0:
        print_clause(clauses_2)
    else:
        print(f"{word2} 언급 조항 없음")

    return clauses_and, clauses_1, clauses_2


In [ ]:
clauses_and, clauses_1, clauses_2 = search_json("/mnt/c/Users/Shic/legal_graph/results/1-2/DCM_1-2_law_main_clause.json", 
                                                "증권신고서", "공개매수신고서")


## main 법령

In [ ]:
import re

def parse_law(text: str, meta_info: dict)->list[dict]:
    law_structure = []
    current_section = {"doc": None, "chapter": None, "section": None, "subsection": None}

    # '편', '장', '절', '관' 패턴
    section_patterns = {
        "doc": re.compile(r'^제/d+편 [^/n]+'),
        "chapter": re.compile(r'^제/d+장 [^/n]+'),
        "section": re.compile(r'^제/d+절 [^/n]+'),
        "subsection": re.compile(r'^제/d+관 [^/n]+')
    }
    
    # 조 패턴 (제X조 or 제X조의Y)
    clause_pattern = re.compile(r'^(제/d+조(?:의/d+)?)(?:/((.*?)/))?/s*(.*)')
    
    # 텍스트 줄 단위로 분할
    lines = text.splitlines()
    current_clause = None
    
    for line in lines:
        line = line.strip()

        # '편', '장', '절', '관' 갱신
        is_section_line = False
        for section, pattern in section_patterns.items():
            if pattern.match(line):
                current_section[section] = line.strip()
                is_section_line = True
                break  # "편", "장", "절", "관" 중 하나를 찾으면, 더 이상 확인하지 않음

        if is_section_line:
            continue  # 섹션 관련 줄은 content에 포함하지 않음

        # 조항 확인
        clause_match = clause_pattern.match(line)
        if clause_match:
            if current_clause:
                # 이전 조항을 저장
                law_structure.append(current_clause)
            # 새 조항 시작
            clause_index = clause_match.group(1)
            subtitle = clause_match.group(2)
            content = clause_match.group(3)
            current_clause = {
                "index": clause_index,
                "subtitle": subtitle,
                "content": content,
                "metadata": {
                    "document_title" : meta_info["document_title"],
                    "date":meta_info["date"],
                    "revise_info":meta_info["revise_info"],
                    "source":meta_info["source"],
                    "title":{
                        "doc": current_section["doc"],
                        "chapter": current_section["chapter"],
                        "section": current_section["section"],
                        "subsection": current_section["subsection"]
                    }
                }
            }
        
        # 내용 추가
        elif current_clause and line:
            if current_clause["content"]:
                current_clause["content"] += "/n" + line
            else:
                current_clause["content"] = line

    if current_clause:
        law_structure.append(current_clause)

    return law_structure



In [ ]:
# 예제 입력 텍스트
main_input_text = """
제1편 총칙

제1조(목적) 이 영은 「자본시장과 금융투자업에 관한 법률」에서 위임된 사항과 그 시행에 관하여 필요한 사항을 규정함을 목적으로 한다.

제3조의2(증권신고서 제출 등의 규제만 적용되는 증권) 법 제4조제1항제2호에서 "대통령령으로 정하는 증권"이란 「상법」에 따른 합자회사ㆍ유한책임회사ㆍ합자조합ㆍ익명조합의 출자지분이 표시된 것을 말한다. 다만, 집합투자증권은 제외한다.
[본조신설 2013. 8. 27.]

제2편 금융투자업

제1장 금융투자업의 인가 및 등록

제1절 인가요건 및 절차

제15조(인가업무 단위 등) ① 법 제12조제1항 각 호 외의 부분에서 "대통령령으로 정하는 업무 단위"란 별표 1과 같다.
② 법 제12조제1항제2호에서 금융투자상품의 범위에 포함되는 증권 중 "대통령령으로 정하는 것"이란 다음 각 호의 것을 말한다.
1. 채무증권
2. 지방채증권
3. 법 제4조제3항에 따른 특수채증권

제4장 영업행위 규칙

제1절 공통 영업행위 규칙

제1관 신의성실의무 등

제42조(상호의 제한) ① 법 제38조제1항에서 "대통령령으로 정하는 문자"란 financial investment(그 한글표기문자를 포함한다)나 그와 비슷한 의미를 가지는 다른 외국어문자(그 한글표기문자를 포함한다)를 말한다. <신설 2009. 2. 3.>
"""

meta_info = {"document_title":"자본시장과 금융투자업에 관한 법률 시행령",
             "date":"2024.08.27",
             "revise_info" : "revised in 2024",
             "source" : "국가법령정보센터"}


In [ ]:
# 파싱 결과 출력
output = parse_law(main_input_text, meta_info)
for entry in output:
    print(entry)


In [ ]:
output[3]

## 부칙 법령

In [ ]:
import re
def parse_supplementary(text: str, meta_info: dict)->list[dict]:
    law_structure = []
    current_section = {"supplementary": None}
    section_patterns = {
        "supplementary" : re.compile("부칙 <제/d+호,/d+/. /d+/. /d+/.>")
    }
        
    # 조 패턴 (제X조 or 제X조의Y)
    clause_pattern = re.compile(r'^(제/d+조(?:의/d+)?)(?:/((.*?)/))?/s*(.*)')
    
    # 텍스트 줄 단위로 분할
    lines = text.splitlines()
    current_clause = None
    
    for line in lines:
        line = line.strip()

        # '편', '장', '절', '관' 갱신
        is_section_line = False
        for section, pattern in section_patterns.items():
            if pattern.match(line):
                current_section[section] = line.strip()
                is_section_line = True
                break  # "편", "장", "절", "관" 중 하나를 찾으면, 더 이상 확인하지 않음

        if is_section_line:
            continue  # 섹션 관련 줄은 content에 포함하지 않음

        # 조항 확인
        clause_match = clause_pattern.match(line)
        if clause_match:
            if current_clause:
                # 이전 조항을 저장
                law_structure.append(current_clause)
            # 새 조항 시작
            clause_index = clause_match.group(1)
            subtitle = clause_match.group(2)
            content = clause_match.group(3)
            current_clause = {
                "index": clause_index,
                "subtitle": subtitle,
                "content": content,
                "metadata": {
                    "document_title" : meta_info["document_title"],
                    "date":meta_info["date"],
                    "revise_info":meta_info["revise_info"],
                    "source":meta_info["source"],
                    "title":{
                        "supplementary": current_section["supplementary"]
                    }
                }
            }
        
        # 내용 추가
        elif current_clause and line:
            if current_clause["content"]:
                current_clause["content"] += "/n" + line
            else:
                current_clause["content"] = line

    if current_clause:
        law_structure.append(current_clause)

    return law_structure

In [2]:
supplementary_input_text = """부칙 <제20947호,2008. 7. 29.>
제1조(시행일) 이 영은 2009년 2월 4일부터 시행한다. 다만, 부칙 제4조는 2008년 8월 4일부터 시행한다.
제2조(다른 법령의 폐지) 다음 각 호의 법령을 각각 폐지한다. 
1. 「증권거래법 시행령」
2. 「선물거래법 시행령」
3. 「간접투자자산 운용업법 시행령」
4. 「신탁업법 시행령」
5. 「종합금융회사에 관한 법률 시행령」
6. 「한국증권선물거래소법 시행령」
제3조 삭제 <2009. 2. 3.>"""

meta_info = {"document_title":"자본시장과 금융투자업에 관한 법률 시행령",
             "date":"2024.08.27",
             "revise_info" : "revised in 2024",
             "source" : "국가법령정보센터"}

In [3]:
articles = parse_supplementary(supplementary_input_text, meta_info)

In [ ]:
articles[0]

## 항 분리

In [ ]:
import re

text = """제68조(불건전 영업행위의 금지) ① 법 제71조 각 호 외의 부분 단서에서 "대통령령으로 정하는 경우"란 다음 각 호의 어느 하나에 해당하는 경우를 말한다. <개정 2012. 6. 29., 2013. 8. 27.>

1. 법 제71조제1호를 적용할 때 다음 각 목의 어느 하나에 해당하는 경우
2. 법 제71조제2호를 적용할 때 다음 각 목의 어느 하나에 해당하는 경우
가. 조사분석자료의 내용이 직접 또는 간접으로 특정 금융투자상품의 매매를 유도하는 것이 아닌 경우
3. 법 제71조제3호를 적용할 때 해당 조사분석자료가 투자자에게 공표되거나 제공되지 아니하고 금융투자업자 내부에서 업무를 수행할 목적으로 작성된 경우
4. 법 제71조제5호를 적용할 때 투자권유대행인 및 투자권유자문인력이 아닌 자에게 금적립계좌등에 대한 투자권유를 하게 하는 경우
② 법 제71조제3호에서 "대통령령으로 정하는 기업금융업무"란 다음 각 호의 어느 하나에 해당하는 업무를 말한다.<개정 2012. 6. 29., 2015. 10. 23., 2021. 10. 21.>
5. 인수업무
6. 모집ㆍ사모ㆍ매출의 주선업무
7. 기업의 인수 및 합병의 중개ㆍ주선 또는 대리업무
8. 기업의 인수ㆍ합병에 관한 조언업무"""

# 정규표현식을 이용해 숫자 특수기호 기준으로 분리
split_text = re.split(r'(?=①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩)', text)

# 숫자 특수기호로 시작하는 텍스트만 포함
filtered_text = [item.strip() for item in split_text if re.match(r'^[①-⑩]', item)]

# 결과 출력
print(filtered_text)


In [ ]:
len(filtered_text)

In [ ]:
filtered_text[1]

In [ ]:
import re

def contains_special_numbers(text):
    # 정규표현식 패턴: ①부터 ⑩까지의 문자
    pattern = r"[①②③④⑤⑥⑦⑧⑨⑩]"
    return bool(re.search(pattern, text))

# 테스트
sample_text = "이 문장에는 이 포함되어 있습니다."
print(contains_special_numbers(sample_text))  # 출력: True


## 정규표현식 테스트

In [11]:
text1 = "제2-1조(청약권유대상자의 수에서 제외되는 자)"
text2 = "제2-2조의3(코넥스시장에 관한 특례 등)"
text3 = "제3조(금융투자상품)"
text4 = "제16조의2(투자매매업 등의 업무 단위 추가 및 등록)"

In [ ]:
import re
clause_pattern = re.compile(r'^(제[/d-]+조(?:의/d+)?)(?:/((.*?)/))?/s*(.*)')

In [ ]:
clause_match = clause_pattern.match(text2)
clause_match.group(1)

In [ ]:
clause_match.group(2)

In [ ]:
import re

def index_to_article(text):
    # '제301조'와 같은 패턴에서 article, article_regulation, subarticle, clause 추출

    # 제2-3조와 같은 패턴에서 article과 article_regulation 추출
    article_match = re.search(r"제(/d+)(?:-(/d+))?조", text)  # 제1조 또는 제2-3조
    subarticle_match = re.search(r"의(/d+)", text)  # 제1조의1
    clause_match = re.search(r"제(/d+)항", text)  # 제1항

    if article_match:
        article = article_match.group(1)
        article_regulation = article_match.group(2)
    else:
        article = None
        article_regulation = None

    subarticle = subarticle_match.group(1) if subarticle_match else None
    clause = clause_match.group(1) if clause_match else None

    return article, article_regulation, subarticle, clause


In [3]:
# 예시 사용
text = "제2-3조의4제5항"
article, article_regulation, subarticle, clause = index_to_article(text)
print("article:", article)
print("article_regulation:", article_regulation)
print("subarticle:", subarticle)
print("clause:", clause)

article: 2
article_regulation: 3
subarticle: 4
clause: 5


In [8]:
text = "제2조의4제5항"
article, article_regulation, subarticle, clause = index_to_article(text)
print("article:", article)
print("article_regulation:", article_regulation)
print("subarticle:", subarticle)
print("clause:", clause)

article: 2
article_regulation: None
subarticle: 4
clause: 5


In [9]:
text = "제2조제5항"
article, article_regulation, subarticle, clause = index_to_article(text)
print("article:", article)
print("article_regulation:", article_regulation)
print("subarticle:", subarticle)
print("clause:", clause)

article: 2
article_regulation: None
subarticle: None
clause: 5


In [11]:
text = "제2조"
article, article_regulation, subarticle, clause = index_to_article(text)
print("article:", article)
print("article_regulation:", article_regulation)
print("subarticle:", subarticle)
print("clause:", clause)

article: 2
article_regulation: None
subarticle: None
clause: None
